# 05 - Continuous Training

After testing, compiling, and uploading the pipeline definition to Cloud Storage, the pipeline is executed with respect to a trigger. We use [Cloud Functions](https://cloud.google.com/functions) and [Cloud Pub/Sub](https://cloud.google.com/pubsub) as a triggering mechanism. The triggering can be scheduled using [Cloud Schedular](https://cloud.google.com/scheduler). The trigger source sends a message to a Cloud Pub/Sub topic that the Cloud Function listens to, and then it submits the pipeline to AI Platform Managed Pipelines to be executed.

This notebook covers the following steps:
1. Create the Cloud Pub/Sub topic.
2. Deploy the Cloud Function 
3. Test triggering a pipeline.
4. Extracting pipeline run metadata.

## Setup

In [ ]:
import json
import os
import logging
import tensorflow as tf
import tfx

logging.getLogger().setLevel(logging.INFO)

print("Tensorflow Version:", tfx.__version__)

In [ ]:
PROJECT = 'ksalama-cloudml' # Change to your project Id.
REGION = 'us-central1'
BUCKET = 'ksalama-cloudml-us' # Change to your bucket.

VERSION = 'v01'
DATASET_DISPLAY_NAME = 'chicago-taxi-tips'
MODEL_DISPLAY_NAME = f'{DATASET_DISPLAY_NAME}-classifier-{VERSION}'
PIPELINE_NAME = f'{MODEL_DISPLAY_NAME}-train-pipeline'

PIPELINES_STORE = f'gs://{BUCKET}/ucaip_demo/compiled_pipelines/'
GCS_PIPELINE_FILE_LOCATION = os.path.join(PIPELINES_STORE, f'{PIPELINE_NAME}.json')
PUBSUB_TOPIC = f'trigger-{PIPELINE_NAME}'
CLOUD_FUNCTION_NAME = f'trigger-{PIPELINE_NAME}-fn'

In [ ]:
!gsutil ls {GCS_PIPELINE_FILE_LOCATION}

## 1. Create a Pub/Sub topic

In [ ]:
!gcloud pubsub topics create {PUBSUB_TOPIC}

## 2. Deploy the Cloud Function

In [ ]:
# Dash separated pipeline parameter names
PARAMETER_NAMES='num_epochs-hidden_units-learning_rate-batch_size'

ENV_VARS=f"""\
PROJECT={PROJECT},\
REGION={REGION},\
GCS_PIPELINE_FILE_LOCATION={GCS_PIPELINE_FILE_LOCATION},\
PARAMETER_NAMES={PARAMETER_NAMES}
"""

!echo {ENV_VARS}

In [ ]:
!rm -r src/pipeline_triggering/.ipynb_checkpoints

In [ ]:
!gcloud functions deploy {CLOUD_FUNCTION_NAME} \
    --region={REGION} \
    --trigger-topic={PUBSUB_TOPIC} \
    --runtime=python37 \
    --source=src/pipeline_triggering\
    --entry-point=trigger_pipeline\
    --stage-bucket={BUCKET}\
    --update-env-vars={ENV_VARS}

## 3. Test Triggering the Pipeline

In [ ]:
from google.cloud import pubsub

publish_client = pubsub.PublisherClient()
topic = f'projects/{PROJECT}/topics/{PUBSUB_TOPIC}'
data = {
    'source_uri': 'pubsub/function/pipline',
    'num_epochs': 7,
    'learning_rate': 0.0015,
    'batch_size': 512,
    'hidden_units': '256,126'
}
message = json.dumps(data)

_ = publish_client.publish(topic, message.encode())

## 4. Extracting Pipeline Runs Metadata

In [ ]:
from src.utils.vertex_utils import VertexClient
vertex_client = VertexClient(PROJECT, REGION)
vertex_client.get_pipeline_metadata_df(pipeline=PIPELINE_NAME)